## Competición Kaggle

#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import urllib.request
import warnings
warnings.filterwarnings('ignore')

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from PIL import Image
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeRegressor

-----------------------------------------------------------------------------------------------------------------

## Exploración y Procesado de los datos

### Data Exploration

In [2]:
df = pd.read_csv("./data/train.csv")
df.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


### Data Processing

In [4]:
df["Company"].unique()

array(['Dell', 'Asus', 'Toshiba', 'MSI', 'HP', 'Lenovo', 'Acer', 'Apple',
       'Fujitsu', 'Mediacom', 'Razer', 'Samsung', 'LG', 'Microsoft',
       'Vero', 'Google', 'Huawei', 'Xiaomi', 'Chuwi'], dtype=object)

In [5]:
df["Product"].unique()

array(['Inspiron 5567', 'X541NA (N4200/4GB/1TB/W10)', 'Portege Z30-C-1CW',
       'Alienware 17', 'Latitude E7270', 'ZenBook Flip',
       'X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux)', 'GP62M 7RDX',
       'UX430UQ-GV209R (i7-7500U/8GB/256GB/GeForce', 'Rog GL553VE-FY052T',
       'X540SA-RBPDN09 (N3710/4GB/1TB/W10)', 'Inspiron 5570',
       'Tecra A50-C-1ZV', '15-cd005nv (A9-9420/6GB/256GB/Radeon',
       'Satellite Pro', 'Latitude 5590', 'Yoga 520-14IKB',
       'Aspire E5-575', 'Thinkpad P51', 'Aspire A517-51G', 'Yoga Book',
       '17-ak002nv (A10-9620P/6GB/2TB/Radeon', 'EliteBook 820',
       'Omen 17-W295', 'Precision M5520',
       'X540UA-DM186 (i3-6006U/4GB/1TB/FHD/Linux)', 'Extensa EX2540-58KR',
       'IdeaPad 310-15IKB', 'IdeaPad 320-15IAP', 'ProBook 640',
       'Inspiron 3576', 'GL553VE-FY082T (i7-7700HQ/8GB/1TB',
       'B51-80 (i7-6500U/8GB/1008GB/Radeon', 'Vostro 3568',
       'Chromebook CB5-571-C1DZ', 'IdeaPad 320-15ABR', 'Inspiron 3552',
       'X553SA-XX021T (N305

In [6]:
df["TypeName"].unique()

array(['Notebook', 'Gaming', 'Ultrabook', '2 in 1 Convertible', 'Netbook',
       'Workstation'], dtype=object)

In [7]:
# Cambiar donde ponga "Netbook" por "Notebook"
df["TypeName"] = df["TypeName"].replace("Netbook", "Notebook")

df["TypeName"].unique()

array(['Notebook', 'Gaming', 'Ultrabook', '2 in 1 Convertible',
       'Workstation'], dtype=object)

In [8]:
df["Inches"].unique()

array([15.6, 13.3, 12.5, 14. , 17.3, 10.1, 11.6, 15.4, 15. , 12. , 13.5,
       12.3, 13. , 13.9, 18.4])

In [9]:
df["ScreenResolution"].unique()

array(['1366x768', 'Full HD 1920x1080', 'IPS Panel 4K Ultra HD 3840x2160',
       'Full HD / Touchscreen 1920x1080', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'IPS Panel Touchscreen 1920x1200',
       '4K Ultra HD / Touchscreen 3840x2160', 'Touchscreen 1366x768',
       'IPS Panel Retina Display 2880x1800', 'IPS Panel 1366x768',
       'IPS Panel Quad HD+ / Touchscreen 3200x1800',
       'Quad HD+ / Touchscreen 3200x1800', '4K Ultra HD 3840x2160',
       'IPS Panel 2560x1440', '1600x900', 'Touchscreen 2560x1440',
       'IPS Panel Quad HD+ 2560x1440',
       'IPS Panel Retina Display 2304x1440', '1440x900',
       'IPS Panel Retina Display 2560x1600', 'Touchscreen 2256x1504',
       '2560x1440', 'IPS Panel Touchscreen / 4K Ultra HD 3840x2160',
       'IPS Panel Touchscreen 2560x1440', '1920x1080',
       'Touchscreen 2400x1600', 'IPS Panel Full HD 2160x1440',
       'Quad HD+ 3200x1800',
       'IPS Panel 4K Ultra HD / Touchscreen 3840x21

In [10]:
# Descartar el tipo de panel porque no todos lo especifican y crear una columna especificando si son de pantalla táctil o no
df["Touchscreen"] = df["ScreenResolution"].apply(lambda x : 1 if "Touchscreen" in x else 0)

# Colocar la nueva columna al lado de la columna "ScreenResolution"
df.insert(df.columns.get_loc("ScreenResolution") + 1, "Touchscreen", df.pop("Touchscreen"))

df.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Touchscreen,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,0,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,0,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,0,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,0,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,1,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [11]:
# Poner todos con la resolución unicamente, ejemplo: 3840x2160
def normalize_resolution(res):
    return res.split()[-1]

df["ScreenResolution"] = df["ScreenResolution"].apply(normalize_resolution)

df["ScreenResolution"].unique()

array(['1366x768', '1920x1080', '3840x2160', '1920x1200', '2880x1800',
       '3200x1800', '2560x1440', '1600x900', '2304x1440', '1440x900',
       '2560x1600', '2256x1504', '2400x1600', '2160x1440'], dtype=object)

In [12]:
df["Cpu"].unique()

array(['Intel Core i7 7500U 2.7GHz',
       'Intel Pentium Quad Core N4200 1.1GHz',
       'Intel Core i5 6200U 2.3GHz', 'Intel Core i7 7700HQ 2.8GHz',
       'Intel Core i5 6300U 2.4GHz', 'Intel Core i3 6006U 2GHz',
       'Intel Pentium Quad Core N3710 1.6GHz',
       'Intel Core i7 8550U 1.8GHz', 'AMD A9-Series A9-9420 3GHz',
       'Intel Pentium Dual Core 4405U 2.1GHz',
       'Intel Core i5 8250U 1.6GHz', 'Intel Core i3 7100U 2.4GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Xeon E3-1535M v6 3.1GHz',
       'Intel Atom x5-Z8550 1.44GHz', 'AMD A10-Series 9620P 2.5GHz',
       'Intel Core i5 7300U 2.6GHz',
       'Intel Celeron Dual Core N3350 1.1GHz',
       'Intel Core i7 6500U 2.5GHz', 'Intel Core i3 6006U 2.0GHz',
       'Intel Celeron Dual Core 3205U 1.5GHz',
       'AMD A12-Series 9720P 3.6GHz',
       'Intel Celeron Dual Core N3050 1.6GHz',
       'Intel Core i3 7130U 2.7GHz', 'Intel Core i5 7300HQ 2.5GHz',
       'Intel Core i5 7200U 2.7GHz', 'Intel Core i7 7820HQ 2.9GHz

In [13]:
# Extraer la velocidad del procesador de la columna Cpu
df["Cpu_Speed"] = df["Cpu"].str.extract(r"(\d+(\.\d+)?)GHz")[0].astype(float)

df["Cpu"] = df["Cpu"].str.replace(r"\s*\d+(\.\d+)?GHz\s*", "", regex = True)

# Colocar la nueva columna al lado de la columna "Cpu"
df.insert(df.columns.get_loc("Cpu") + 1, "Cpu_Speed", df.pop("Cpu_Speed"))

df.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Touchscreen,Cpu,Cpu_Speed,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,0,Intel Core i7 7500U,2.7,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,0,Intel Pentium Quad Core N4200,1.1,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,1920x1080,0,Intel Core i5 6200U,2.3,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,3840x2160,0,Intel Core i7 7700HQ,2.8,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,1920x1080,1,Intel Core i5 6300U,2.4,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [14]:
df["Ram"].unique()

array(['8GB', '4GB', '16GB', '12GB', '6GB', '32GB', '2GB', '24GB'],
      dtype=object)

In [15]:
# Quitar "GB" de todos
df["Ram"] = df["Ram"].str.replace("GB", "")

# Convertir de object a número entero
df["Ram"] = df["Ram"].astype(int)

df["Ram"].unique()

array([ 8,  4, 16, 12,  6, 32,  2, 24])

In [16]:
df["Memory"].unique()

array(['1TB HDD', '256GB SSD', '256GB SSD +  1TB HDD',
       '512GB SSD +  2TB HDD', '128GB SSD +  1TB HDD', '128GB SSD',
       '1TB SSD', '64GB Flash Storage', '2TB HDD', '500GB HDD',
       '1.0TB Hybrid', '16GB Flash Storage', '256GB Flash Storage',
       '180GB SSD', '32GB Flash Storage', '512GB SSD',
       '512GB SSD +  1TB HDD', '256GB SSD +  500GB HDD',
       '256GB SSD +  2TB HDD', '1TB SSD +  1TB HDD', '64GB SSD',
       '128GB Flash Storage', '256GB SSD +  1.0TB Hybrid', '32GB SSD',
       '512GB Flash Storage', '240GB SSD', '256GB SSD +  256GB SSD',
       '16GB SSD', '8GB SSD', '64GB Flash Storage +  1TB HDD',
       '1.0TB HDD', '1TB HDD +  1TB HDD', '512GB SSD +  512GB SSD',
       '128GB SSD +  2TB HDD', '32GB HDD', '512GB SSD +  1.0TB Hybrid'],
      dtype=object)

In [17]:
# Separar y sumar sólo los números, quedando las cifras en GB
def extract_memory_gb(memory):
    matches = re.findall(r"(\d+)\s*(TB|GB)", memory)
    total_gb = 0
    for match in matches:
        num, unit = match
        if unit == "TB":
            total_gb += int(num) * 1000
        elif unit == "GB":
            total_gb += int(num)
    return total_gb

df["Memory_GB"] = df["Memory"].apply(extract_memory_gb)

# Colocar las nuevas columnas en la posición de la columna "Memory"
df.insert(df.columns.get_loc("Memory") + 1, "Memory_GB", df.pop("Memory_GB"))

# Eliminar la columna "Memory"
df = df.drop(columns = "Memory")

df.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Touchscreen,Cpu,Cpu_Speed,Ram,Memory_GB,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,0,Intel Core i7 7500U,2.7,8,1000,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,0,Intel Pentium Quad Core N4200,1.1,4,1000,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,1920x1080,0,Intel Core i5 6200U,2.3,8,256,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,3840x2160,0,Intel Core i7 7700HQ,2.8,16,1256,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,1920x1080,1,Intel Core i5 6300U,2.4,8,256,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [18]:
df["Gpu"].unique()

array(['AMD Radeon R7 M445', 'Intel HD Graphics 505',
       'Intel HD Graphics 520', 'Nvidia GeForce GTX 1070',
       'Nvidia GeForce GT 940MX', 'Nvidia GeForce GTX 1050',
       'Nvidia GeForce 940MX', 'Nvidia GeForce GTX 1050 Ti',
       'Intel HD Graphics 405', 'Intel UHD Graphics 620',
       'AMD Radeon 530', 'Intel HD Graphics 510', 'Intel HD Graphics 620',
       'Nvidia Quadro M2200M', 'Nvidia GeForce MX150',
       'Intel HD Graphics 400', 'Nvidia Quadro M1200',
       'Nvidia GeForce 920MX', 'Intel HD Graphics 500', 'AMD Radeon 520',
       'AMD Radeon R5 M330', 'AMD Radeon R5 M420X', 'Intel HD Graphics',
       'Intel HD Graphics 630', 'AMD Radeon R5 M430',
       'Intel Iris Pro Graphics', 'Nvidia GeForce GTX 940MX',
       'Intel HD Graphics 515', 'Nvidia GeForce GTX 1080',
       'AMD Radeon R2 Graphics', 'Nvidia GeForce 150MX', 'AMD Radeon R5',
       'Nvidia GeForce GTX 1050Ti', 'AMD Radeon R9 M385',
       'Nvidia GeForce 930MX', 'Nvidia GeForce GTX 1060',
       'Nv

In [19]:
df["OpSys"].unique()

array(['Windows 10', 'Windows 7', 'Linux', 'Chrome OS', 'No OS',
       'Mac OS X', 'Windows 10 S', 'macOS', 'Android'], dtype=object)

In [20]:
df["Weight"].unique()

array(['2.36kg', '2kg', '1.2kg', '4.42kg', '1.26kg', '2.26kg', '2.4kg',
       '1.3kg', '2.5kg', '2.65kg', '2.02kg', '2.2kg', '1.95kg', '1.75kg',
       '1.88kg', '1.74kg', '2.23kg', '3kg', '0.69kg', '2.71kg', '3.35kg',
       '1.78kg', '2.13kg', '2.32kg', '2.18kg', '2.20kg', '2.1kg', '1.7kg',
       '2.3kg', '1.91kg', '2.31kg', '1.4kg', '1.48kg', '2.04kg', '1.54kg',
       '1.86kg', '1.98kg', '1.47kg', '2.67kg', '1.85kg', '1.5kg',
       '0.97kg', '1.6kg', '1.68kg', '2.09kg', '2.33kg', '3.49kg',
       '3.78kg', '1.36kg', '2.77kg', '1.29kg', '1.62kg', '1.05kg',
       '2.05kg', '1.9kg', '1.28kg', '1.49kg', '2.62kg', '2.0kg', '1.63kg',
       '1.37kg', '2.73kg', '2.69kg', '1.25kg', '4.14kg', '2.9kg',
       '1.23kg', '2.37kg', '1.10kg', '1.90kg', '2.63kg', '1.39kg',
       '2.8kg', '2.94kg', '1.43kg', '1.22kg', '2.21kg', '1.32kg', '2.6kg',
       '1.45kg', '1.14kg', '4.36kg', '1.8kg', '1.1kg', '3.3kg', '3.2kg',
       '4.2kg', '1.35kg', '2.19kg', '1.96kg', '4.0kg', '1.84kg', '1.65kg',


In [21]:
# Quitar "kg" de todos
df["Weight"] = df["Weight"].str.replace("kg", "")

# Convertir de object a número entero
df["Weight"] = df["Weight"].astype(float)

df["Weight"].unique()

array([2.36 , 2.   , 1.2  , 4.42 , 1.26 , 2.26 , 2.4  , 1.3  , 2.5  ,
       2.65 , 2.02 , 2.2  , 1.95 , 1.75 , 1.88 , 1.74 , 2.23 , 3.   ,
       0.69 , 2.71 , 3.35 , 1.78 , 2.13 , 2.32 , 2.18 , 2.1  , 1.7  ,
       2.3  , 1.91 , 2.31 , 1.4  , 1.48 , 2.04 , 1.54 , 1.86 , 1.98 ,
       1.47 , 2.67 , 1.85 , 1.5  , 0.97 , 1.6  , 1.68 , 2.09 , 2.33 ,
       3.49 , 3.78 , 1.36 , 2.77 , 1.29 , 1.62 , 1.05 , 2.05 , 1.9  ,
       1.28 , 1.49 , 2.62 , 1.63 , 1.37 , 2.73 , 2.69 , 1.25 , 4.14 ,
       2.9  , 1.23 , 2.37 , 1.1  , 2.63 , 1.39 , 2.8  , 2.94 , 1.43 ,
       1.22 , 2.21 , 1.32 , 2.6  , 1.45 , 1.14 , 4.36 , 1.8  , 3.3  ,
       3.2  , 4.2  , 1.35 , 2.19 , 1.96 , 4.   , 1.84 , 1.65 , 1.76 ,
       1.17 , 1.09 , 2.59 , 4.33 , 2.7  , 1.64 , 0.92 , 1.34 , 4.6  ,
       1.56 , 1.13 , 2.08 , 2.43 , 2.38 , 1.93 , 1.252, 1.16 , 1.24 ,
       1.38 , 2.75 , 1.87 , 1.42 , 0.99 , 2.14 , 1.44 , 3.14 , 3.4  ,
       2.24 , 1.15 , 1.83 , 3.21 , 2.06 , 2.83 , 1.19 , 1.31 , 1.11 ,
       1.27 , 0.98 ,

In [22]:
# Eliminar las columnas "laptop_ID" y "Product"
df = df.drop(columns = ["laptop_ID", "Product"])

In [23]:
df.head()

,id,Company,TypeName,Inches,ScreenResolution,Touchscreen,Cpu,Cpu_Speed,Ram,Memory_GB,Gpu,OpSys,Weight,Price_euros
0,268,Dell,Notebook,15.6,1366x768,0,Intel Core i7 7500U,2.7,8,1000,AMD Radeon R7 M445,Windows 10,2.36,749.00
1,347,Asus,Notebook,15.6,1366x768,0,Intel Pentium Quad Core N4200,1.1,4,1000,Intel HD Graphics 505,Windows 10,2.00,449.00
2,353,Toshiba,Notebook,13.3,1920x1080,0,Intel Core i5 6200U,2.3,8,256,Intel HD Graphics 520,Windows 7,1.20,1460.00
3,578,Dell,Gaming,15.6,3840x2160,0,Intel Core i7 7700HQ,2.8,16,1256,Nvidia GeForce GTX 1070,Windows 10,4.42,2868.99
4,42,Dell,Ultrabook,12.5,1920x1080,1,Intel Core i5 6300U,2.4,8,256,Intel HD Graphics 520,Windows 7,1.26,1713.37


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   Company           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Touchscreen       912 non-null    int64  
 6   Cpu               912 non-null    object 
 7   Cpu_Speed         912 non-null    float64
 8   Ram               912 non-null    int32  
 9   Memory_GB         912 non-null    int64  
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    float64
 13  Price_euros       912 non-null    float64
dtypes: float64(4), int32(1), int64(3), object(6)
memory usage: 96.3+ KB


-----------------------------------------------------------------------------------------------------------------

## Modelado

### Definir Target

In [25]:
target = "Price_euros"

### Transformar categóricas

In [26]:
df = pd.get_dummies(df, columns = ["Company", "TypeName", "ScreenResolution", "Cpu", "Gpu", "OpSys"])

### Definir X e y

In [27]:
X = df.drop([target], axis = 1)
y = df[target]

### Dividir X_train, X_test, y_train, y_test

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Crear y entrenar el modelo


In [29]:
model_names = ["Regresion Lineal", "DecisionTree", "Random Forest", "LightGBM", "CatBoost"]
lr_reg = LinearRegression()
tree_reg = DecisionTreeRegressor(random_state = 42)
rf_reg = RandomForestRegressor(random_state = 42)
lgb_reg = LGBMRegressor(random_state = 42, verbose = -100)
cat_reg = CatBoostRegressor(random_state = 42, verbose = False)

model_set = [lr_reg, tree_reg, rf_reg, lgb_reg, cat_reg]

In [30]:
metricas_cv = {}
valores = []
for nombre,modelo in zip(model_names, model_set):
    print(modelo)
    if nombre == "Regresion Lineal":
        metricas_cv[nombre] = cross_val_score(modelo, X_train, y_train, cv = 3, scoring = "neg_mean_squared_error")
    else:
        metricas_cv[nombre] = cross_val_score(modelo, X_train, y_train, cv = 3, scoring = "neg_mean_squared_error")
    valores.append(np.mean(metricas_cv[nombre]))
ganador = list(metricas_cv.keys())[np.argmax(valores)]

LinearRegression()
DecisionTreeRegressor(random_state=42)
RandomForestRegressor(random_state=42)
LGBMRegressor(random_state=42, verbose=-100)


In [31]:
for model_name, valores in metricas_cv.items():
    print(f"Model <{model_name}>, RMSE_CV: {np.sqrt(-np.mean(valores))}")
print(f"El ganador es {ganador}")

Model <Regresion Lineal>, RMSE_CV: 359.2428970882599
Model <DecisionTree>, RMSE_CV: 414.78147643544105
Model <Random Forest>, RMSE_CV: 309.72938846879225
Model <LightGBM>, RMSE_CV: 370.91374401840085
Model <CatBoost>, RMSE_CV: 302.3010759935606
El ganador es CatBoost


In [ ]:
"""
# Random Forest
params_grid = {
    "n_estimators" : [100, 200, 300],
    "max_depth" : [20, 25, 30],
    "min_samples_leaf": [0.5, 1, 1.5],
    "max_features": ["sqrt", "log2", None],
    "max_samples": [0.3, 0.6, 1]
}

rf_grid = GridSearchCV(rf_reg,
                       param_grid = params_grid,
                       cv = 5,
                       scoring = "neg_mean_squared_error",
                       n_jobs = -1)

rf_grid.fit(X_train, y_train)
"""

In [32]:
# CatBoost
params_grid= {
    "depth" : [3, 6, 12],
    "learning_rate" : [0.1, 0.2, 0.3],
    "colsample_bylevel" : [0.25, 0.35, 0.45],
    "iterations" : [1000, 1250, 1500],
    "border_count" : [100, 125, 150]
}

cat_grid = GridSearchCV(cat_reg,
                       param_grid = params_grid,
                       cv = 5,
                       scoring = "neg_mean_squared_error",
                       n_jobs = -1)

cat_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x0000026D476267D0>,
             n_jobs=-1,
             param_grid={'border_count': [100, 125, 150],
                         'colsample_bylevel': [0.25, 0.35, 0.45],
                         'depth': [3, 6, 12], 'iterations': [1000, 1250, 1500],
                         'learning_rate': [0.1, 0.2, 0.3]},
             scoring='neg_mean_squared_error')

In [33]:
cat_grid.best_params_

{'border_count': 125,
 'colsample_bylevel': 0.35,
 'depth': 6,
 'iterations': 1250,
 'learning_rate': 0.2}

### Sacar métricas y valorar el modelo

In [34]:
y_pred = cat_grid.best_estimator_.predict(X_test)

print("Error absoluto medio (MAE):", mean_absolute_error(y_test, y_pred))
print("Error porcentual absoluto medio (MAPE):", mean_absolute_percentage_error(y_test, y_pred) * 100)
print("Raíz del error cuadrático medio (RMSE):", np.sqrt(mean_squared_error(y_test, y_pred)))

Error absoluto medio (MAE): 190.423819798
Error porcentual absoluto medio (MAPE): 19.51269993190799
Raíz del error cuadrático medio (RMSE): 300.02929818844495


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

In [ ]:
"""
# Random Forest
modelo_final = rf_grid.fit(X, y)
"""

In [35]:
# CatBoost
modelo_final = cat_grid.fit(X, y)

### 2. Carga los datos de `test.csv` para predecir.

In [36]:
df_test = pd.read_csv("./data/test.csv")
df_test.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,181,1098,HP,Spectre x360,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.3kg
1,708,330,Acer,Aspire 5,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8GB,256GB SSD,AMD Radeon RX 540,Windows 10,2.2kg
2,862,1260,Acer,Aspire ES1-572,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,500GB HDD,Intel HD Graphics 520,Linux,2.4kg
3,1064,1137,HP,EliteBook 1040,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.43kg
4,702,1015,HP,ENVY -,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.34kg


In [37]:
# Cambiar donde ponga "Netbook" por "Notebook"
df_test["TypeName"] = df_test["TypeName"].replace("Netbook", "Notebook")

# Descartar tipo de panel + crear columna "Touchscreen" + recolocar nueva columna + dejar unicamente la resolución
df_test["Touchscreen"] = df_test["ScreenResolution"].apply(lambda x : 1 if "Touchscreen" in x else 0)
df_test.insert(df_test.columns.get_loc("ScreenResolution") + 1, "Touchscreen", df_test.pop("Touchscreen"))

def normalize_resolution(res):
    return res.split()[-1]
df_test["ScreenResolution"] = df_test["ScreenResolution"].apply(normalize_resolution)

# Extraer velocidad del procesador + crear columna "Cpu_Speed" + recolocar nueva columna
df_test["Cpu_Speed"] = df_test["Cpu"].str.extract(r"(\d+(\.\d+)?)GHz")[0].astype(float)
df_test["Cpu"] = df_test["Cpu"].str.replace(r"\s*\d+(\.\d+)?GHz\s*", "", regex = True)
df_test.insert(df_test.columns.get_loc("Cpu") + 1, "Cpu_Speed", df_test.pop("Cpu_Speed"))

# Quitar "GB" de todos + convertir a numérico
df_test["Ram"] = df_test["Ram"].str.replace("GB", "")
df["Ram"] = df["Ram"].astype(int)

# Dejar sólo un valor numérico en GB + crear columna "Memory_GB" + recolocar nueva columna + eliminar la columna "Memory"
def extract_memory_gb(memory):
    matches = re.findall(r"(\d+)\s*(TB|GB)", memory)
    total_gb = 0
    for match in matches:
        num, unit = match
        if unit == "TB":
            total_gb += int(num) * 1000
        elif unit == "GB":
            total_gb += int(num)
    return total_gb
df_test["Memory_GB"] = df_test["Memory"].apply(extract_memory_gb)

df_test.insert(df_test.columns.get_loc("Memory") + 1, "Memory_GB", df_test.pop("Memory_GB"))
df_test = df_test.drop(columns = "Memory")

# Quitar "kg" de todos + convertir a numérico
df_test["Weight"] = df_test["Weight"].str.replace("kg", "")
df["Weight"] = df["Weight"].astype(float)

# Eliminar las columnas "laptop_ID" y "Product"
df_test = df_test.drop(columns = ["laptop_ID", "Product"])

# Convertir variables categóricas en variables dummy
df_test = pd.get_dummies(df_test, columns = ["Company", "TypeName", "ScreenResolution", "Cpu", "Gpu", "OpSys"])

In [38]:
# Identificar las columnas que faltan en df_test
missing_columns = set(X.columns) - set(df_test.columns)

# Agregar las columnas faltantes a df_test con valor 0
for column in missing_columns:
    df_test[column] = 0

# Confirmar que las columnas estan en el mismo orden que en X
df_test = df_test[X.columns]

In [39]:
df_test.shape

(391, 228)

**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [40]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [41]:
predictions = modelo_final.predict(df_test)
predictions

array([2235.2593931 ,  749.05789977,  392.7438353 , 1503.97359348,
       1063.49985038, 1271.37844016, 2649.52287132,  501.65139886,
       2520.71300043, 2202.30786679, 1303.62098805,  543.70310723,
        380.99562863,  735.87140572,  365.60351296, 1390.28420593,
        595.04020176, 1953.87784958, 1540.07199806, 1600.40624369,
       1744.57221827, 1414.95598256, 1510.24050469, 1431.39003893,
       1826.34058056,  432.50778833,  922.67275832,  370.02249714,
       1105.90854718,  526.53219342,  849.42615813, 1507.5541026 ,
       2001.49321456,  354.93374012,  595.49345256, 1661.2791511 ,
        804.21014963,  541.13447516, 2975.11889112,  614.53762902,
        668.87013681,  767.8624762 , 1253.80840026, 1768.46466199,
       1012.36964774, 1729.00212683,  445.42613372, 3617.46193468,
        192.11754713, 1250.67026884, 1074.64441805,  907.53312101,
        739.98453089,  356.56095149, 1244.29625074,  782.41559316,
        685.52173597,  242.06220406, 1418.9267726 , 1134.42740

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [42]:
sample = pd.read_csv("./data/sample_submission.csv") # Esta es mi ruta del archivo, la vuestra puede ser otra

In [43]:
sample.head()

,id,Price_euros
0,1014,752.0
1,845,499.0
2,1151,1747.0
3,1265,245.0
4,573,1179.0


In [44]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [45]:
submission = pd.DataFrame({"id" : df_test["id"], "Price_euros" : predictions})

submission.to_csv("predictions.csv", index = False)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [46]:
def chequeator(df_to_submit):

    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")

In [47]:
chequeator(submission) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

You're ready to submit!
